# Tutorial: Ammonia Population at Risk (PAR) Analysis

This notebook is a production-grade, ready-to-run walkthrough for **Population at Risk (PAR) assessment** of an ammonia release using `pyELDQM`.

It covers:
- scenario configuration (source, weather, AEGL thresholds, population raster)
- Gaussian dispersion simulation and AEGL zone extraction
- raster-based population counting per zone using `compute_par_counts_from_raster`
- risk classification (CRITICAL / HIGH / MODERATE–LOW) against configurable PAR thresholds
- geographic statistics (area, density, max distance) per AEGL zone
- interactive Folium dispersion map displayed inline

## 1) Runtime Notes

This notebook supports two execution modes:
1. **Installed package mode** (`pip install pyeldqm`)
2. **Workspace mode** (running inside the project repository)

The import cell automatically detects and handles both.

In [17]:
from __future__ import annotations
import sys
from math import radians, cos, sin, asin, sqrt
from pathlib import Path
from datetime import datetime
import numpy as np
from IPython.display import display, Markdown, HTML

IMPORT_MODE = "package"
try:
    from pyeldqm.core.dispersion_models.gaussian_model import calculate_gaussian_dispersion
    from pyeldqm.core.meteorology.stability import get_stability_class
    from pyeldqm.core.chemical_database import ChemicalDatabase
    from pyeldqm.core.visualization.folium_maps import (
        create_dispersion_map,
        meters_to_latlon,
        add_facility_markers,
        calculate_optimal_zoom_level,
    )
    from pyeldqm.core.utils.features import setup_computational_grid
    from pyeldqm.core.utils.zone_extraction import extract_zones
    from pyeldqm.app.utils.population import compute_par_counts_from_raster
except ModuleNotFoundError:
    IMPORT_MODE = "workspace"
    cwd = Path.cwd().resolve()
    candidates = [cwd] + list(cwd.parents)
    project_root = next(
        (p for p in candidates if (p / "core").exists() and (p / "app").exists()), None
    )
    if project_root is None:
        raise RuntimeError(
            "Could not resolve project root. Run from repository root or install pyeldqm via pip."
        )
    if str(project_root) not in sys.path:
        sys.path.insert(0, str(project_root))
    from core.dispersion_models.gaussian_model import calculate_gaussian_dispersion
    from core.meteorology.stability import get_stability_class
    from core.chemical_database import ChemicalDatabase
    from core.visualization.folium_maps import (
        create_dispersion_map,
        meters_to_latlon,
        add_facility_markers,
        calculate_optimal_zoom_level,
    )
    from core.utils.features import setup_computational_grid
    from core.utils.zone_extraction import extract_zones
    from app.utils.population import compute_par_counts_from_raster

display(Markdown(f"**Import mode:** `{IMPORT_MODE}`"))

**Import mode:** `workspace`

## 2) Scenario Configuration

Update these values to model your own scenario.

In [18]:
# Population raster
RASTER_PATH = (
    "D:/OneDrive - UET/After PhD/Research/pyELDQM/pyELDQM/"
    "data/population/data/population/pak_pop_2026_CN_100m_R2025A_v1.tif"
)
CRITICAL_THRESHOLD = 10000.0          # PAR >= this -> CRITICAL
HIGH_THRESHOLD = 5000.0               # PAR >= this -> HIGH
# Chemical
CHEMICAL_NAME = "AMMONIA"
MOLECULAR_WEIGHT = 17.03              # g/mol
# Release location
SOURCE_LAT = 31.6911
SOURCE_LON = 74.0822
TIMEZONE_OFFSET_HRS = 5.0
# Release parameters
RELEASE_TYPE = "single"               # single | multi
SOURCE_TERM_MODE = "continuous"       # continuous | instantaneous
RELEASE_RATE = 800.0                  # g/s (continuous)
TANK_HEIGHT = 3.0                     # m
DURATION_MINUTES = 30.0               # min
MASS_RELEASED_KG = 500.0              # kg (instantaneous mode)
TERRAIN_ROUGHNESS = "URBAN"           # URBAN | RURAL
RECEPTOR_HEIGHT_M = 1.5               # m
# Weather
WEATHER_MODE = "manual"
WIND_SPEED = 2.5                      # m/s
WIND_DIRECTION = 90.0                 # degrees
TEMPERATURE_C = 25.0                  # C
HUMIDITY = 65.0                       # %
CLOUD_COVER = 30.0                    # %
# AEGL thresholds (ppm)
AEGL_THRESHOLDS = {
    "AEGL-1": 30.0,
    "AEGL-2": 160.0,
    "AEGL-3": 1100.0,
}
# Computational grid
X_MAX = 12000
Y_MAX = 4000
NX = 500
NY = 500

assert NX > 10 and NY > 10, "Grid resolution is too low."
assert X_MAX > 0 and Y_MAX > 0, "Grid extents must be positive."
assert WIND_SPEED > 0, "Wind speed must be > 0."
print("Scenario configured successfully.")
print(f"Raster               : {Path(RASTER_PATH).name}")
print("All outputs will be shown inline in this notebook.")

Scenario configured successfully.
Raster               : pak_pop_2026_CN_100m_R2025A_v1.tif
All outputs will be shown inline in this notebook.


## 3) Helper Functions

Distance utility for reporting maximum zone reach from the source.

In [19]:
def haversine_km(lat1, lon1, lat2, lon2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * asin(sqrt(a)) * 6371.0


def max_distance_from_source_km(polygon, src_lat, src_lon):
    if polygon is None or polygon.is_empty:
        return None
    max_d = 0.0
    try:
        for lon, lat in polygon.exterior.coords:
            d = haversine_km(src_lat, src_lon, lat, lon)
            if d > max_d:
                max_d = d
    except Exception:
        return None
    return max_d if max_d > 0 else None

## 4) Chemical Properties

Fetch core records from the chemical database for transparency and traceability.

In [20]:
print("=" * 72)
print("CHEMICAL PROPERTIES")
print("=" * 72)
try:
    db = ChemicalDatabase()
    chem_data = next(
        (c for c in db.get_all_chemicals() if c.get("name") == CHEMICAL_NAME),
        None,
    )
    if chem_data:
        fields = [
            ("Name",                  "name"),
            ("Molecular Weight",      "molecular_weight"),
            ("Boiling Point (K)",     "boiling_point_K"),
            ("Density (kg/m3)",       "density_kgm3"),
            ("Vapor Pressure (kPa)",  "vapor_pressure_kPa"),
            ("IDLH (ppm)",            "idlh_ppm"),
            ("LC50 (ppm)",            "lc50_ppm"),
            ("AEGL-1 (60 min)",       "aegl1_60min"),
            ("AEGL-2 (60 min)",       "aegl2_60min"),
            ("AEGL-3 (60 min)",       "aegl3_60min"),
            ("ERPG-1",                "erpg1"),
            ("ERPG-2",                "erpg2"),
            ("ERPG-3",                "erpg3"),
        ]
        for label, key in fields:
            value = chem_data.get(key)
            if value not in (None, "", 0):
                print(f"{label:<26}: {value}")
    else:
        print(f"Name                     : {CHEMICAL_NAME}")
        print(f"Molecular Weight         : {MOLECULAR_WEIGHT} g/mol")
        print("Database record not found; using configured molecular weight.")
except Exception as ex:
    print(f"Name                     : {CHEMICAL_NAME}")
    print(f"Molecular Weight         : {MOLECULAR_WEIGHT} g/mol")
    print(f"Database query failed     : {ex}")

CHEMICAL PROPERTIES
Name                      : AMMONIA
Molecular Weight          : 17.03
AEGL-1 (60 min)           : 30 ppm
AEGL-2 (60 min)           : 160 ppm
AEGL-3 (60 min)           : 1100 ppm
ERPG-1                    : 25
ERPG-2                    : 150
ERPG-3                    : 750


## 5) Run Dispersion and Extract AEGL Zones

In [21]:
simulation_datetime = datetime.now()
weather = {
    "wind_speed":    WIND_SPEED,
    "wind_dir":      WIND_DIRECTION,
    "temperature_K": TEMPERATURE_C + 273.15,
    "humidity":      HUMIDITY / 100.0,
    "cloud_cover":   CLOUD_COVER / 100.0,
    "source":        "manual",
}
stability_class = get_stability_class(
    wind_speed=weather["wind_speed"],
    datetime_obj=simulation_datetime,
    latitude=SOURCE_LAT,
    longitude=SOURCE_LON,
    cloudiness_index=weather.get("cloud_cover", 0) * 10,
    timezone_offset_hrs=TIMEZONE_OFFSET_HRS,
)
X, Y, _, _ = setup_computational_grid(x_max=X_MAX, y_max=Y_MAX, nx=NX, ny=NY)
release_duration_s = DURATION_MINUTES * 60.0
source_q = RELEASE_RATE
dispersion_mode = "continuous" if SOURCE_TERM_MODE == "continuous" else "instantaneous"
concentration, U_local, resolved_stability, resolved_sources = calculate_gaussian_dispersion(
    weather=weather,
    X=X,
    Y=Y,
    source_lat=SOURCE_LAT,
    source_lon=SOURCE_LON,
    molecular_weight=MOLECULAR_WEIGHT,
    default_release_rate=source_q,
    default_height=TANK_HEIGHT,
    z_ref=3.0,
    z_measurement=RECEPTOR_HEIGHT_M,
    t=release_duration_s,
    t_r=release_duration_s,
    mode=dispersion_mode,
    sources=[{
        "lat": SOURCE_LAT, "lon": SOURCE_LON,
        "name": "Release Source",
        "height": TANK_HEIGHT, "rate": source_q,
        "color": "red",
    }],
    latitude=SOURCE_LAT,
    longitude=SOURCE_LON,
    timezone_offset_hrs=TIMEZONE_OFFSET_HRS,
    roughness=TERRAIN_ROUGHNESS,
    datetime_obj=simulation_datetime,
)
threat_zones = extract_zones(
    X, Y, concentration, AEGL_THRESHOLDS,
    SOURCE_LAT, SOURCE_LON, wind_dir=weather["wind_dir"],
)
print(f"Simulation time       : {simulation_datetime}")
print(f"Resolved wind speed   : {U_local:.2f} m/s")
print(f"Stability class       : {resolved_stability}")
print(f"Grid shape            : {concentration.shape}")

Simulation time       : 2026-02-23 22:09:14.121887
Resolved wind speed   : 2.50 m/s
Stability class       : F
Grid shape            : (500, 500)


## 6) Report Zone Distances

Maximum distance is measured from source to polygon boundary.

In [22]:
print("=" * 72)
print("METEOROLOGICAL CONDITIONS")
print("=" * 72)
print(f"Wind Speed            : {weather['wind_speed']:.2f} m/s")
print(f"Wind Direction        : {weather['wind_dir']:.0f} deg")
print(f"Temperature           : {weather['temperature_K'] - 273.15:.1f} C")
print(f"Humidity              : {weather['humidity'] * 100:.0f} %")
print(f"Cloud Cover           : {weather['cloud_cover'] * 100:.0f} %")
print(f"Stability Class       : {resolved_stability}")
print()
print("=" * 72)
print("AEGL THREAT ZONE DISTANCES FROM SOURCE")
print("=" * 72)
for zone_name in ["AEGL-3", "AEGL-2", "AEGL-1"]:
    poly = threat_zones.get(zone_name)
    threshold = AEGL_THRESHOLDS.get(zone_name, "N/A")
    threshold_str = f"{threshold:.0f} ppm" if isinstance(threshold, (int, float)) else str(threshold)
    dist_km = max_distance_from_source_km(poly, SOURCE_LAT, SOURCE_LON) if poly else None
    if dist_km is not None:
        print(
            f"{zone_name:<8} threshold: {threshold_str:<10} max distance: {dist_km:.3f} km ({dist_km * 1000:.0f} m)"
        )
    else:
        print(f"{zone_name:<8} threshold: {threshold_str:<10} max distance: -- (no zone formed)")

METEOROLOGICAL CONDITIONS
Wind Speed            : 2.50 m/s
Wind Direction        : 90 deg
Temperature           : 25.0 C
Humidity              : 65 %
Cloud Cover           : 30 %
Stability Class       : F

AEGL THREAT ZONE DISTANCES FROM SOURCE
AEGL-3   threshold: 1100 ppm   max distance: -- (no zone formed)
AEGL-2   threshold: 160 ppm    max distance: 0.622 km (622 m)
AEGL-1   threshold: 30 ppm     max distance: 1.769 km (1769 m)


## 7) Calculate Population at Risk (PAR)

`compute_par_counts_from_raster` clips the GeoTIFF raster to each AEGL zone polygon
and sums the population pixels within that zone.

In [23]:
print(f"Raster file           : {Path(RASTER_PATH).name}")
par_counts, par_note = compute_par_counts_from_raster(RASTER_PATH, threat_zones)

aegl3_pop = par_counts.get("AEGL-3", 0)
aegl2_pop = par_counts.get("AEGL-2", 0)
aegl1_pop = par_counts.get("AEGL-1", 0)
total_par = aegl3_pop + aegl2_pop + aegl1_pop

print("=" * 72)
print("POPULATION AT RISK BY AEGL ZONE")
print("=" * 72)
print(f"AEGL-3 (life-threatening) : {aegl3_pop:>10,}  people")
print(f"AEGL-2 (irreversible)     : {aegl2_pop:>10,}  people")
print(f"AEGL-1 (mild effects)     : {aegl1_pop:>10,}  people")
print("-" * 72)
print(f"Total PAR                 : {total_par:>10,}  people")
print(f"Note                      : {par_note}")
print("=" * 72)

Raster file           : pak_pop_2026_CN_100m_R2025A_v1.tif
POPULATION AT RISK BY AEGL ZONE
AEGL-3 (life-threatening) :          0  people
AEGL-2 (irreversible)     :        174  people
AEGL-1 (mild effects)     :        403  people
------------------------------------------------------------------------
Total PAR                 :        577  people
Note                      : PAR calculated from selected population raster.


## 8) Risk Classification and Geographic Statistics

In [24]:
# Risk classification
if total_par >= CRITICAL_THRESHOLD:
    risk_label = "CRITICAL"
elif total_par >= HIGH_THRESHOLD:
    risk_label = "HIGH"
else:
    risk_label = "MODERATE / LOW"

print("=" * 72)
print("RISK CLASSIFICATION")
print("=" * 72)
print(f"Total PAR             : {total_par:,} people")
print(f"Risk level            : {risk_label}")
print(f"Critical threshold    : >= {CRITICAL_THRESHOLD:,.0f} people")
print(f"High threshold        : >= {HIGH_THRESHOLD:,.0f} people")
print()

# Geographic statistics
try:
    from shapely.geometry import Point as _Pt
    _src = _Pt(SOURCE_LON, SOURCE_LAT)

    print("=" * 72)
    print("GEOGRAPHIC STATISTICS BY AEGL ZONE")
    print("=" * 72)
    print(f"{'Zone':<8} {'Area (km2)':>10} {'Max dist (km)':>14} {'Population':>12} {'Density (p/km2)':>16}")
    print("-" * 72)
    for zone_name in ["AEGL-3", "AEGL-2", "AEGL-1"]:
        poly = threat_zones.get(zone_name)
        if poly is None or poly.is_empty:
            print(f"{zone_name:<8} {'--':>10} {'--':>14} {'--':>12} {'--':>16}")
            continue
        b = poly.bounds
        mid_lat = (b[1] + b[3]) / 2
        area_km2 = max(
            ((b[3] - b[1]) * 111) * ((b[2] - b[0]) * 111 * np.cos(np.radians(mid_lat))),
            0.0,
        )
        max_d_km = max(
            _src.distance(_Pt(coord)) * 111 for coord in poly.exterior.coords
        )
        pop = par_counts.get(zone_name, 0)
        density = pop / area_km2 if area_km2 > 0 else 0
        print(f"{zone_name:<8} {area_km2:>10.2f} {max_d_km:>14.3f} {pop:>12,} {density:>16,.0f}")
    print("=" * 72)
except Exception as ex:
    print(f"Geographic statistics unavailable: {ex}")

RISK CLASSIFICATION
Total PAR             : 577 people
Risk level            : MODERATE / LOW
Critical threshold    : >= 10,000 people
High threshold        : >= 5,000 people

GEOGRAPHIC STATISTICS BY AEGL ZONE
Zone     Area (km2)  Max dist (km)   Population  Density (p/km2)
------------------------------------------------------------------------
AEGL-3           --             --           --               --
AEGL-2         0.02          0.729          174            7,547
AEGL-1         0.18          2.075          403            2,233


## 9) Build PAR Map (Inline)

In [25]:
lat_grid, lon_grid = meters_to_latlon(X, Y, SOURCE_LAT, SOURCE_LON, weather["wind_dir"])
zoom_level, bounds = calculate_optimal_zoom_level(
    lat_grid,
    lon_grid,
    concentration,
    threshold=min(AEGL_THRESHOLDS.values()),
)
m = create_dispersion_map(
    source_lat=SOURCE_LAT,
    source_lon=SOURCE_LON,
    x_grid=X,
    y_grid=Y,
    concentration=concentration,
    thresholds=AEGL_THRESHOLDS,
    wind_direction=weather["wind_dir"],
    zoom_start=zoom_level,
    chemical_name=CHEMICAL_NAME,
    source_height=TANK_HEIGHT,
    wind_speed=weather["wind_speed"],
    stability_class=resolved_stability,
    include_heatmap=True,
    include_compass=True,
)
if resolved_sources:
    m = add_facility_markers(m, resolved_sources) or m
if bounds[0] is not None:
    try:
        m.fit_bounds([[bounds[1], bounds[3]], [bounds[0], bounds[2]]], padding=(0.1, 0.1), max_zoom=18)
    except Exception:
        pass
print("PAR threat map generated. Displaying inline below.")

PAR threat map generated. Displaying inline below.


## 10) Display PAR Map Inline

In [26]:
if 'm' in globals() and m is not None:
    display(HTML(m._repr_html_()))
else:
    print("Map object not found. Run Cell 19 first.")

## 11) Inline Results Summary

Quick run summary for tutorial users.

In [27]:
peak_ppm = float(np.nanmax(concentration))
zone_stats = {}
for zone_name in ["AEGL-3", "AEGL-2", "AEGL-1"]:
    poly = threat_zones.get(zone_name)
    zone_stats[zone_name] = max_distance_from_source_km(poly, SOURCE_LAT, SOURCE_LON) if poly else None
print("Completed successfully.")
print(f"Chemical              : {CHEMICAL_NAME}")
print(f"Peak concentration    : {peak_ppm:.2f} ppm")
print(f"Total PAR             : {total_par:,} people")
print(f"Risk level            : {risk_label}")
print("Max zone distances:")
for z, d in zone_stats.items():
    if d is None:
        print(f"  {z}: no zone formed")
    else:
        print(f"  {z}: {d:.3f} km ({d * 1000:.0f} m)")

Completed successfully.
Chemical              : AMMONIA
Peak concentration    : 852.65 ppm
Total PAR             : 577 people
Risk level            : MODERATE / LOW
Max zone distances:
  AEGL-3: no zone formed
  AEGL-2: 0.622 km (622 m)
  AEGL-1: 1.769 km (1769 m)


## Troubleshooting

- If imports fail, run from repository root **or** install package mode: `pip install pyeldqm`.

- If the map does not render inline, rerun Cell 19 then Cell 21.

- If zones are empty, increase release rate/duration, reduce thresholds, or expand the grid extent.

- If the raster file is not found, update `RASTER_PATH` in Cell 5 to the correct absolute path
  for your local installation. The raster must be a GeoTIFF with population counts per pixel.

- PAR counts of zero with zones present usually indicates a CRS mismatch between the raster
  and the zone polygons — verify the raster projection matches WGS-84 (EPSG:4326).